# A Baseline Named Entity Recognizer for Twitter

In this notebook I'll follow the example presented in [Named entities and random fields](http://www.orbifold.net/default/2017/06/29/dutch-ner/) to train a conditional random field to recognize named entities in Twitter data. The data and some of the code below are taken from a programming assignment in the amazing class [Natural Language Processing](https://www.coursera.org/learn/language-processing) offered by [Coursera](https://www.coursera.org/). In the assignment we were shown how to build a named entity recognizer using deep learning with a bidirectional LSTM, which is a pretty complicated approach and I wanted to have a baseline model to see what sort of accuracy should be expected on this data.

### 1. Preparing the Data

First load the text and tags for training, validation and test data:

In [1]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token
            if token.startswith("http://") or token.startswith("https://"): token = "<URL>"
            elif token.startswith("@"): token = "<USR>"
            tweet_tokens.append(token)
            tweet_tags.append(tag) 
    return tokens, tags
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

The CRF model uses part of speech tags as features so we'll neet to add those to the datasets.

In [2]:
%%time
import nltk

def build_sentence(tokens, tags):
    pos_tags = [item[-1] for item in nltk.pos_tag(tokens)]
    return list(zip(tokens, pos_tags, tags))

def build_sentences(tokens_set, tags_set):
    return [build_sentence(tokens, tags) for tokens, tags in zip(tokens_set, tags_set)]

train_sents = build_sentences(train_tokens, train_tags)
validation_sents = build_sentences(validation_tokens, validation_tags)
test_sents = build_sentences(test_tokens, test_tags)

CPU times: user 7.06 s, sys: 192 ms, total: 7.26 s
Wall time: 7.26 s


### 2. Computing Features

In [3]:

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i - 1][0]
        postag1 = sent[i - 1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        postag1 = sent[i + 1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for token, postag, label in sent]


def sent2tokens(sent):
    return [token for token, postag, label in sent]


X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

### 3. Train the Model

In [4]:
import sklearn_crfsuite

In [5]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.12,
    c2=0.01,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.12, c2=0.01,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

### 4. Evaluate the Model

In [6]:
from sklearn_crfsuite import metrics

In [7]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-musicartist',
 'I-musicartist',
 'B-product',
 'I-product',
 'B-company',
 'B-person',
 'B-other',
 'I-other',
 'B-facility',
 'I-facility',
 'B-sportsteam',
 'B-geo-loc',
 'I-geo-loc',
 'I-company',
 'I-person',
 'B-movie',
 'I-movie',
 'B-tvshow',
 'I-tvshow',
 'I-sportsteam']

In [8]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

/Users/rflagg/anaconda3/envs/intro-dl/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.52923408198771571

In [9]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

               precision    recall  f1-score   support

    B-company      0.857     0.571     0.686        84
    I-company      0.875     0.525     0.656        40
   B-facility      0.758     0.532     0.625        47
   I-facility      0.816     0.656     0.727        61
    B-geo-loc      0.855     0.606     0.709       165
    I-geo-loc      0.786     0.423     0.550        52
      B-movie      0.000     0.000     0.000         8
      I-movie      0.000     0.000     0.000        10
B-musicartist      0.333     0.074     0.121        27
I-musicartist      0.667     0.083     0.148        24
      B-other      0.580     0.388     0.465       103
      I-other      0.397     0.290     0.335        93
     B-person      0.691     0.538     0.605       104
     I-person      0.655     0.545     0.595        66
    B-product      0.500     0.143     0.222        28
    I-product      0.564     0.367     0.444        60
 B-sportsteam      0.909     0.323     0.476        31
 I-sports

/Users/rflagg/anaconda3/envs/intro-dl/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 5. Tuning Parameters

I tried tuning the parameters c1 and c2 of the model using [randomized grid search](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) but was not able to improve the results that way.  I plan to try [GPyOpt](https://github.com/SheffieldML/GPyOpt) to see if that will do better but don't have time to do that here.